In [1]:
print("Ok")

Ok


In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

extracted_data=load_pdf_file(data='../data_rp/')

In [22]:
print(type(extracted_data[0]))

<class 'langchain_core.documents.base.Document'>


In [20]:
# Function to clean text
import re
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords", quiet=True)
def clean_text(text):
    """
    Cleans the extracted text by:
    - Removing non-alphanumeric characters (except spaces).
    - Normalizing whitespace.
    - Removing headers, footers, or page numbers using regex patterns.
    """
    # Remove non-alphanumeric characters (except spaces)
    # print(text)
    print("Before Preprocessing: ")
    print(text[:200])
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Normalize whitespace (remove extra spaces, tabs, and newlines)

    # Remove page numbers or headers/footers (example pattern)
    text = re.sub(r"Page \d+|Header Text|Footer Text", "", text)
    # print(text)
    stop_words = set(stopwords.words("english"))
    pattern = r'\b(' + '|'.join(map(re.escape, stop_words)) + r')\b'

    result = re.sub(pattern, '', text)
    text = re.sub(r"\s+", " ", result).strip()
    print()
    print("After Preprocessing: ")
    print(text[:200])
    return text

# Process each page and clean its content
cleaned_pages = []
for page in extracted_data:
    if page.page_content:  # Ensure the page is not blank
        # print(page.page_content[:200])
        page.page_content = clean_text(page.page_content)
        # print(page.page_content[:200])

# Print cleaned content for verification
# for cleaned_page in cleaned_pages:
#     print(f"Page {cleaned_page['page_number']}:\n{cleaned_page['content']}\n")

Before Preprocessing: 
Tevatron: An Efficient and Flexible Toolkit for Dense Retrieval
Luyu Gao1, Xueguang Ma2, Jimmy Lin2, and Jamie Callan1
1Language Technologies Institute
Carnegie Mellon University
2David R. Cheriton Sc

After Preprocessing: 
Tevatron An Efficient Flexible Toolkit Dense Retrieval Luyu Gao1 Xueguang Ma2 Jimmy Lin2 Jamie Callan1 1Language Technologies Institute Carnegie Mellon University 2David R Cheriton School Computer Sci
Before Preprocessing: 
The positive targets are usually human judged and the negative
texts are usually non-relevant texts from top results of a baseline
retrieval system such as BM25. The second format (not shown due
{
"qu

After Preprocessing: 
The positive targets usually human judged negative texts usually nonrelevant texts top results baseline retrieval system BM25 The second format shown due queryid query id query query text positivepass
Before Preprocessing: 
library [11] as our retriever’s backend. It implements several ef-
ficient indice

In [31]:
import re
from langchain_core.documents import Document

def section_based_chunking(extracted_data):
    chunks = []
    
    # Pattern to identify section headers in academic papers
    section_pattern = re.compile(r'\n\d+[\.\s]+[A-Z][\w\s]+\n|^\d+[\.\s]+[A-Z][\w\s]+\n|\n[A-Z][A-Z\s]+\n')
    
    for doc in extracted_data:
        text = doc.page_content
        
        # Find all section boundaries
        section_matches = list(section_pattern.finditer(text))
        
        if not section_matches:
            # If no sections found, use default chunking
            splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
            doc_chunks = splitter.split_text(text)
        else:
            # Split by identified sections
            doc_chunks = []
            for i in range(len(section_matches)):
                start = section_matches[i].start()
                end = section_matches[i+1].start() if i < len(section_matches)-1 else len(text)
                section_text = text[start:end]
                doc_chunks.append(section_text)
        
        # Create document objects with metadata
        for chunk in doc_chunks:
            chunks.append(Document(page_content=chunk, metadata=doc.metadata))
    
    return chunks

text_chunks=section_based_chunking(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 27


In [28]:
from langchain.text_splitter import MarkdownTextSplitter

def text_split_markdown(extracted_data):
    markdown_splitter = MarkdownTextSplitter(chunk_size=300, chunk_overlap=50)
    
    # Convert documents to markdown format if needed
    markdown_chunks = []
    for doc in extracted_data:
        # Process each document
        chunks = markdown_splitter.create_documents([doc.page_content])
        # Preserve metadata
        for chunk in chunks:
            chunk.metadata = doc.metadata
        markdown_chunks.extend(chunks)
    
    return markdown_chunks
text_chunks=text_split_markdown(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 122


In [29]:
# #Split the Data into Text Chunks
# def text_split(extracted_data):
#     text_splitter=RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
#     text_chunks=text_splitter.split_documents(extracted_data)
#     return text_chunks

# text_chunks=text_split(extracted_data)
# print("Length of Text Chunks", len(text_chunks))

In [32]:
print("Text Chunks Generated: ")
print("------------------------")
print("Chunk-0: ")
print(text_chunks[0])
print()
print("Chunk-1: ")
print(text_chunks[1])
print()
print("Chunk-2: ")
print(text_chunks[2])

Text Chunks Generated: 
------------------------
Chunk-0: 
page_content='
ABSTRACT
Recent rapid advancements in deep pre-trained language models
and the introductions of large datasets have powered research in
embedding-based dense retrieval. While several good research pa-
pers have emerged, many of them come with their own software
stacks. These stacks are typically optimized for some particular re-
search goals instead of efficiency or code structure. In this paper, we
present Tevatron, a dense retrieval toolkit optimized for efficiency,
flexibility, and code simplicity. Tevatron provides a standardized
pipeline for dense retrieval including text processing, model train-
ing, corpus/query encoding, and search. This paper presents an
overview of Tevatron and demonstrates its effectiveness and effi-
ciency across several IR and QA data sets. We also show how Teva-
tron’s flexible design enables easy generalization across datasets,
model architectures, and accelerator platforms(GPU/TPU

In [41]:
print(text_chunks[6])

page_content='HFQueryDataset and HFCorpusDataset for encoding) which will
perform fast parallel data formatting and tokenization. Processed
data is internally represented as a datasets.Dataset object and
is stored in Apache Arrow format which can be memory-mapped
and randomly accessed by offset.' metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with acmart 2018/07/25 v1.55 Typesetting articles for the Association for Computing Machinery and hyperref 2020-05-15 v7.00e Hypertext links for LaTeX', 'creationdate': '2022-03-14T00:37:09+00:00', 'author': 'Luyu Gao1, Xueguang Ma2, Jimmy Lin2, and Jamie Callan1', 'keywords': '', 'moddate': '2022-03-14T00:37:09+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': 'Tevatron: An Efficient and Flexible Toolkit for Dense Retrieval', 'trapped': '/False', 'source': '../data_rp/RP1.pdf', 'total_pages': 5, 'page': 1, 'page_label': '2'}


In [36]:
# for t in text_chunks[:3]:
#     print(clean_text(t.page_content))
#     print("----------------------------")
#     print()

clean_text("Command-line interface. Section 4 shows the experimental results of running Tevatron with various models and datasets. 2 TOOLKIT OVERVIEW Tevatron is packaged as a Python module available on the Python Package Index. Tevatron can be installed via pip, as follows: \$ pip install tevatron==0.1.0")

Before Preprocessing: 
Command-line interface. Section 4 shows the experimental results of running Tevatron with various models and datasets. 2 TOOLKIT OVERVIEW Tevatron is packaged as a Python module available on the Python Package Index. Tevatron can be installed via pip, as follows: \$ pip install tevatron==0.1.0

After Preprocessing: 


<>:6: SyntaxWarning: invalid escape sequence '\$'
<>:6: SyntaxWarning: invalid escape sequence '\$'
/var/folders/yr/ccrg_j914ln8wydhc308crs00000gp/T/ipykernel_28948/1705238954.py:6: SyntaxWarning: invalid escape sequence '\$'
  clean_text("Command-line interface. Section 4 shows the experimental results of running Tevatron with various models and datasets. 2 TOOLKIT OVERVIEW Tevatron is packaged as a Python module available on the Python Package Index. Tevatron can be installed via pip, as follows: \$ pip install tevatron==0.1.0")


'Commandline interface Section 4 shows experimental results running Tevatron various models datasets 2 TOOLKIT OVERVIEW Tevatron packaged Python module available Python Package Index Tevatron installed via pip follows pip install tevatron010'

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

/var/folders/yr/ccrg_j914ln8wydhc308crs00000gp/T/ipykernel_28948/3587992455.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Users/AjaysPC1/Documents/LangChain_Projects/research_paperbot/mchatbot_venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [39]:
print(query_result)

[-0.03447726368904114, 0.03102319873869419, 0.006734949070960283, 0.0261089988052845, -0.03936203196644783, -0.16030246019363403, 0.06692399084568024, -0.0064414506778120995, -0.047450508922338486, 0.014758870005607605, 0.07087535411119461, 0.05552756413817406, 0.01919330097734928, -0.02625131793320179, -0.010109501890838146, -0.026940520852804184, 0.022307423874735832, -0.022226650267839432, -0.14969263970851898, -0.01749308779835701, 0.007676230277866125, 0.054352328181266785, 0.003254461335018277, 0.031725965440273285, -0.08462142199277878, -0.029405977576971054, 0.05159567669034004, 0.0481240376830101, -0.003314798232167959, -0.05827917903661728, 0.04196927323937416, 0.022210724651813507, 0.12818880379199982, -0.022338908165693283, -0.011656265705823898, 0.06292837113142014, -0.032876305282115936, -0.09122606366872787, -0.031175317242741585, 0.052699554711580276, 0.04703483358025551, -0.0842030718922615, -0.030056171119213104, -0.020744813606142998, 0.009517783299088478, -0.0037217

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY2')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [10]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "notebook-paper-llm-db5"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': '93a4ec6fa1120030981f2c7d27930539', 'date': 'Fri, 28 Mar 2025 20:39:05 GMT', 'server': 'Google Frontend', 'Content-Length': '196', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"FORBIDDEN","message":"Request failed. You've reached the max serverless indexes allowed in project Default (5). To add more serverless indexes, upgrade your plan."},"status":403}


In [13]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [11]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [12]:
# Load Existing index 

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
docsearch

In [38]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})
retrieved_docs = retriever.invoke("Is it compatible with Pytorch?")
retrieved_docs

[Document(id='be7f7c39-5a01-4bd4-9a93-e3120136ffa7', metadata={'page': 4.0, 'source': '../data_rp/RP1.pdf'}, page_content='Alban Desmaison, Andreas Kopf, Edward Yang, Zachary DeVito, Martin Raison,\nAlykhan Tejani, Sasank Chilamkurthy, Benoit Steiner, Lu Fang, Junjie Bai, and\nSoumith Chintala. 2019. PyTorch: An Imperative Style, High-Performance Deep'),
 Document(id='2c3ae5de-4207-4b9e-8970-507af14a5707', metadata={'page': 4.0, 'source': '../data_rp/RP1.pdf'}, page_content='Chintala 2019 PyTorch An Imperative Style HighPerformance Deep Learning Library In Advances Neural Information Processing Systems 32 H Wallach H Larochelle A Beygelzimer F dAlchBuc E Fox R Garnett Eds Curran Associates Inc 80248035 httppapersneuripsccpaper'),
 Document(id='c6a769c6-0c40-4f2c-92c5-bcd48479eca9', metadata={'page': 0.0, 'source': '../data_rp/RP1.pdf'}, page_content='To accommodate different research needs, we select two deep\nlearning frameworks for Tevatron, Pytorch [21] and JAX[4]. Py-\ntorch’s eage

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",
                             temperature=0.3)

In [16]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [17]:
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [19]:
response = rag_chain.invoke({"input": "what is Tevatron?"})
print(response["answer"])

Tevatron is a Python module available on the Python Package Index and can be installed via pip.  It supports the full pipeline of data preparation, training, encoding, and search for dense retrieval.  More information can be found at tevatron.ai.



In [20]:
response = rag_chain.invoke({"input": "How to use the toolkit?"})
print(response["answer"])

Tevatron is a Python module available on the Python Package Index.  It can be installed via pip: `pip install tevatron==0.1.0`.  The toolkit has a modularized design and command-line interfaces for development.

